<a href="https://colab.research.google.com/github/rayvoelker/cincy-py-2022-06-30/blob/main/2022_06_39_cincy_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thursday, June 30, 2022

## CincyPy Reconnect!

<img src="https://raw.githubusercontent.com/rayvoelker/cincy-py-2022-06-30/main/cincy_py.jpg">

Ray Voelker | ray.voelker@gmail.com

### Data Vis

1. Hamiltion County Ohio Board of Elections -- Voter lists:
   - https://votehamiltoncountyohio.gov/campaign-media/voter-lists/

1. U.S. Census Bureau -- The 2020 ZIP Code Tabulation Areas (ZCTAs) (Polygons With All Geocodes)
   - https://www.census.gov/geographies/mapping-files/2020/geo/tiger-line-file.html

In [1]:
# NOTE: pandas, and altair are already included with the base colab image...
# !pip install -U pandas --quiet
# !pip install -U altair --quiet

!pip install -U duckdb --quiet

import pandas as pd
import altair as alt

from google.colab import data_table
# enable the formatter for more functionality with tabular data
data_table.enable_dataframe_formatter()

In [2]:
# get the current Hamilton County registerd voter list ...
# downloaded and compressed from here: https://votehamiltoncountyohio.gov/campaign-media/voter-lists/

!wget --quiet https://github.com/rayvoelker/cincy-py-2022-06-30/raw/main/VoterListExport-20220627-no.tar.xz --output-document=VoterListExport.tar.xz

In [3]:
import duckdb

# connect to an in-memory database
con = duckdb.connect()

df = pd.read_csv(
    "VoterListExport.tar.xz", 
    parse_dates=['RegisteredDate'],
    low_memory=False
)

In [4]:
# take a look at the columns in the voter list dataframe ...
df.columns

Index(['VoterListExport-20220627-no.csv', 'PrecinctNum', 'PrecinctSplit',
       'PrecinctName', 'RegisteredDate', 'FirstName', 'MiddleName', 'LastName',
       'SuffixName', 'MiddleName.1', 'SuffixName.1', 'Phone', 'Status',
       'PartyCode', 'BirthYear', 'AddressPreDirectional', 'AddressNumber',
       'AddressFraction', 'AddressStreet', 'AddressSuffix', 'AddressOther',
       'CityName', 'AddressZip', 'Congress', 'Senate', 'House', 'Judicial',
       'School', 'CountySchool', 'VocationalSchool', 'RegistrationDate',
       'GENERAL_NOV_2022', 'PRIMARY_AUG_2022', 'PRIMARY_MAY_2022',
       'GENERAL_NOV_2021', 'GENERAL_NOV_2020', 'PRIMARY_MARCH_2020',
       'GENERAL_NOV_2019', 'GENERAL_NOV_2018', 'PRIMARY_MAY_2018',
       'GENERAL_NOV_2017'],
      dtype='object')

In [5]:
# casually download ~500 MB of shape data ... 
# we'll get to this later ... 
!wget --quiet https://www2.census.gov/geo/tiger/TIGER2021/ZCTA520/tl_2021_us_zcta520.zip --output-document=tl_2021_us_zcta520.zip

# if the above doesn't work, the file is also in this repo:
# !wget --quiet https://github.com/rayvoelker/cincy-py-2022-06-30/raw/main/tl_2021_us_zcta520.tar.xz --output-document=tl_2021_us_zcta520.tar.xz

In [6]:
# DuckDB allows us to efficently run SQL on the Pandas DataFrame!
# https://duckdb.org/2021/05/14/sql-on-pandas.html
result_df = con.execute("""\
  SELECT
    AddressNumber,
    AddressStreet,
    AddressSuffix,
    CityName,
    AddressZip,
    BirthYear,
    PartyCode,
    PrecinctNum,
    PrecinctName,
    RegisteredDate,
    strftime('%Y', RegisteredDate) as reg_year, 
    Status, 
    PRIMARY_MAY_2022
  FROM
    df
  ORDER BY RANDOM()
  LIMIT 1000
  """
).df()

result_df

,AddressNumber,AddressStreet,AddressSuffix,CityName,AddressZip,BirthYear,PartyCode,PrecinctNum,PrecinctName,RegisteredDate,reg_year,Status,PRIMARY_MAY_2022
0,1725.0,CLENEAY,AVE,CINCINNATI,45212.0,1998.0,NON,4411.0,NORWOOD 1-A,2020-11-03,2020,A,NaN
1,320.0,GLENSPRINGS,DR,CINCINNATI,45246.0,1956.0,NON,5207.0,SPRINGDALE G,1989-07-17,1989,A,NaN
2,9101.0,WINTON,RD,CINCINNATI,45231.0,1937.0,NON,8726.0,SPRINGFIELD Z,1989-07-17,1989,I,NaN
3,11834.0,COLERAIN,AVE,CINCINNATI,45252.0,1956.0,NON,5901.0,COLERAIN A,2014-11-04,2014,A,NaN
4,654.0,FLAGSTAFF,DR,CINCINNATI,45215.0,1976.0,NON,5305.0,WYOMING E,1995-10-14,1995,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2012.0,WESTWOOD NORTHERN,BLVD,CINCINNATI,45225.0,1930.0,NON,2203.0,CINCINNATI 22-C,1989-07-17,1989,I,NaN
996,7825.0,AFFINITY,PL,CINCINNATI,45231.0,1948.0,DEM,4111.0,MT HEALTHY 1-A,1989-07-17,1989,A,P-DEM
997,945.0,FINDLAY,ST,CINCINNATI,45214.0,1975.0,NON,1801.0,CINCINNATI 18-A,2020-10-05,2020,A,NaN
998,82.0,JACOB,DR,HARRISON,45030.0,1946.0,NON,6402.0,CROSBY B,1992-03-09,1992,A,NaN


In [7]:
# What is the voter status by year of registration?
# Query:
#
result_df = con.execute("""\
  SELECT
  strftime('%Y', RegisteredDate) as 'Registraion Year', 
  Status, 
  count(*) as count_voters
  FROM
  df
  WHERE
  'Registraion Year' IS NOT NULL
  AND 'Status' IS NOT NULL
  GROUP BY 1,2
  HAVING count(*) > 1
  ORDER BY 3 DESC
""").df()

result_df

,Registraion Year,Status,count_voters
0,1989,A,103105
1,2020,A,42222
2,2016,A,27754
3,2004,A,22860
4,2018,A,21819
...,...,...,...
71,1986,A,14
72,1967,A,3
73,1970,I,2
74,1988,I,2


In [8]:
# What is the voter status by year of registration?
# Chart:
#
alt.Chart(result_df).mark_bar().encode(
    x=alt.X('count_voters', ), # sort='-y'
    y=alt.Y('Registraion Year', sort='-x'), # sort='-x'
    tooltip=['Registraion Year', 'count_voters', 'Status'],
    color=alt.Color(
        'Status',
        scale=alt.Scale(
            # https://vega.github.io/vega/docs/schemes/
            # scheme='set1'
            scheme='set2'
        )
    )
).properties(
    title=''
).configure_title(
    # fontSize=20,
    anchor='start',
)

alt.Chart(...)

In [9]:
# What are the party codes in the data?
result_df = con.execute("""\
  SELECT
  lower(PartyCode) as party_code,
  count(*) as count_voters
  FROM
  df
  WHERE
  PartyCode IS NOT NULL
  GROUP BY 
  1
  ORDER BY 2 DESC
""").df()

result_df

,party_code,count_voters
0,non,431157
1,dem,85813
2,rep,72755
3,lib,100
4,u,1
5,con,1


In [10]:
# What is the voter status by PartyCode?
# Query:
#
result_df = con.execute("""\
  SELECT
  -- strftime('%Y', RegisteredDate) as 'Registraion Year',
  lower(PartyCode) as party_code,
  -- CASE PartyCode
  --    WHEN 'non' THEN 'NON'
  --    ELSE PartyCode
  --END as PartyCode,
  -- 'FirstName', 'MiddleName', 'LastName', 'SuffixName',
  -- 'MiddleName.1', 'SuffixName.1', 'Phone', 
  Status, 
  count(*) as count_voters
  FROM
  df
  WHERE
  lower(PartyCode) IN (
    SELECT
    lower(PartyCode) as party_code
    FROM
    df
    WHERE
    PartyCode IS NOT NULL
    GROUP BY 
    1
    HAVING count(*) >= 100
  )
  GROUP BY 
  1,2
""").df()

result_df

,party_code,Status,count_voters
0,non,I,109132
1,non,A,321940
2,dem,A,84595
3,rep,A,72456
4,rep,I,294
5,dem,I,1210
6,lib,A,98
7,dem,M,8
8,non,M,85
9,rep,M,5


In [11]:
# What is the voter status by PartyCode?
# Chart:
#
alt.Chart(result_df).mark_bar().encode(
    x=alt.X('count_voters', ),
    y=alt.Y('party_code', sort='-x'),
    tooltip=['party_code', 'count_voters', 'Status'],
    color=alt.Color(
        'Status',
        scale=alt.Scale(
            # https://vega.github.io/vega/docs/schemes/
            scheme='dark2'
        )
    )
).properties(
    title='Voter Status by PartyCode'
).configure_title(
    # fontSize=20,
    anchor='middle',
)

alt.Chart(...)

In [12]:
# What percentage of registered voters voted in the 2022 Primary Election?

# Query:
#
result_df = con.execute("""\
  WITH count_data as (
    SELECT
    PRIMARY_MAY_2022,
    count(*) as count_voters
    from
    df
    GROUP BY 1
  )
  SELECT
  PRIMARY_MAY_2022,
  ROUND(
    ((count_voters * 1.0 ) / (SELECT SUM(count_voters) FROM count_data)) * 100.0, 
    2
  ) as pct
  FROM
  count_data
""").df()

result_df

,PRIMARY_MAY_2022,pct
0,NaN,83.37
1,P-DEM,5.41
2,P-REP,8.42
3,A-REP,1.09
4,A-DEM,1.56
5,P-NON,0.13
6,A-NON,0.02


In [13]:
# What percentage of registered voters voted in the 2022 Primary Election?

alt.Chart(result_df).mark_bar().encode(
    x=alt.X('pct', ),
    # y=alt.Y('party_code', sort='-x'),
    tooltip=['PRIMARY_MAY_2022', 'pct',],
    color=alt.Color(
        'PRIMARY_MAY_2022',
        scale=alt.Scale(
            # https://vega.github.io/vega/docs/schemes/
            scheme='accent'
        )
    )
).properties(
    title='Percent Voters Voting in PRIMARY_MAY_2022',
    width=600,
    # height=300
).configure_title(
    # fontSize=20,
    anchor='middle',
)

alt.Chart(...)

In [14]:
result_df = con.execute("""\
  SELECT
    AddressZip,
    count(PRIMARY_MAY_2022) AS count_voters
  FROM
    df
  WHERE
    PRIMARY_MAY_2022 IS NOT NULL
    AND AddressZip IS NOT NULL
  GROUP BY 1
""").df()

result_df

,AddressZip,count_voters
0,45231.0,5046
1,45215.0,4112
2,45237.0,2374
3,45224.0,2787
4,45246.0,1747
5,45209.0,1187
6,45244.0,2716
7,45202.0,1790
8,45241.0,2832
9,45208.0,3621


In [15]:
!pip install -U geopandas --quiet
!pip install -U folium matplotlib mapclassify --quiet
# !pip install -U pyshp
# !pip install -U shapely
# !pip install -U descartes
# !pip install -U osmnx

In [16]:
import pandas as pd
import geopandas as gpd
import duckdb

# connect to an in-memory database
con = duckdb.connect()

df = pd.read_csv(
    "VoterListExport.tar.xz", 
    parse_dates=['RegisteredDate'],
    low_memory=False
)

gdf_org = gpd.read_file('tl_2021_us_zcta520.zip')

In [17]:
result_df = con.execute("""\
  SELECT
    cast(AddressZip as INTEGER) as AddressZip,
    count(PRIMARY_MAY_2022) AS count_voters
  FROM
    df
  WHERE
    PRIMARY_MAY_2022 IS NOT NULL
    AND AddressZip IS NOT NULL
  GROUP BY 1
  HAVING count(PRIMARY_MAY_2022) >= 5
""").df()


# merge the above dataframe and the dataframe containing our 
# GEOID20
result_df.rename(columns={"AddressZip": "GEOID20"}, inplace=True)
# convert the column to string so we can do our merge
result_df['GEOID20'] = result_df['GEOID20'].astype(str)

gdf = gdf_org.merge(result_df, on='GEOID20', how='inner')

gdf[
    gdf['GEOID20'].isin(
      [str(zip_code) for zip_code in result_df['GEOID20']]
    )
].explore(
    "count_voters",
    cmap="YlGn"
)

In [18]:
# Hamilton County Ohio Voters in the 2020 General Elections

result_df = con.execute("""\
  SELECT
    cast(AddressZip as INTEGER) as AddressZip,
    count(GENERAL_NOV_2020) AS count_voters
  FROM
    df
  WHERE
    GENERAL_NOV_2020 IS NOT NULL
    AND AddressZip IS NOT NULL
  GROUP BY 1
  HAVING count(GENERAL_NOV_2020) >= 5
""").df()


# merge the above dataframe and the dataframe containing our 
# GEOID20
result_df.rename(columns={"AddressZip": "GEOID20"}, inplace=True)
# convert the column to string so we can do our merge
result_df['GEOID20'] = result_df['GEOID20'].astype(str)

gdf = gdf_org.merge(result_df, on='GEOID20', how='inner')

gdf[
    gdf['GEOID20'].isin(
      [str(zip_code) for zip_code in result_df['GEOID20']]
    )
].explore(
    "count_voters",
    cmap="YlGn"
)

In [19]:
result_df = con.execute("""\
  SELECT
  -- PRIMARY_MARCH_2016,
  lower(PartyCode) as PartyCode,
  cast(AddressZip as INTEGER) as AddressZip,
  count(*) AS count_voters
  FROM
  df
  WHERE
  lower(PartyCode) = 'rep'
  AND AddressZip is not null
  GROUP BY 1,2

""").df()

# merge the above dataframe and the dataframe containing our 
# GEOID20
result_df.rename(columns={"AddressZip": "GEOID20"}, inplace=True)
# convert the column to string so we can do our merge
result_df['GEOID20'] = result_df['GEOID20'].astype(str)

gdf = gdf_org.merge(result_df, on='GEOID20', how='inner')

gdf[
    gdf['GEOID20'].isin(
      [str(zip_code) for zip_code in result_df['GEOID20']]
    )
].explore(
    "count_voters",
    cmap="Reds"
)

In [20]:
result_df = con.execute("""\
  SELECT
  -- PRIMARY_MARCH_2016,
  lower(PartyCode) as PartyCode,
  cast(AddressZip as INTEGER) as AddressZip,
  count(*) AS count_voters
  FROM
  df
  WHERE
  lower(PartyCode) = 'dem'
  AND AddressZip is not null
  GROUP BY 1,2

""").df()

# merge the above dataframe and the dataframe containing our 
# GEOID20
result_df.rename(columns={"AddressZip": "GEOID20"}, inplace=True)
# convert the column to string so we can do our merge
result_df['GEOID20'] = result_df['GEOID20'].astype(str)

gdf = gdf_org.merge(result_df, on='GEOID20', how='inner')

gdf[
    gdf['GEOID20'].isin(
      [str(zip_code) for zip_code in result_df['GEOID20']]
    )
].explore(
    "count_voters",
    cmap="Blues"
)